# Funciones descarga GCM
***

_Autor:_    __Salvador Navas__ <br> _Revisión:_ __27/02/2020__ <br>


__Descripción__:<br>
https://portal.nccs.nasa.gov/datashare/

<font color='indianred'>Ejecutar con python de Ubuntu Windows 10.</font>

__Cosas a corregir__ <br>

In [3]:
#import libraries
from pylab import *
import scipy.io
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap, cm
from datetime import datetime, timedelta
from scipy.io import netcdf
#from netCDF4 import Dataset 
import sys
#sys.path.append('/home/javi/source/libraries_py')
import h5py
import numpy as np
import pandas as pd
import glob
import os
import csv as csv
import urllib
import tqdm
import time
sys.version

'3.7.3 | packaged by conda-forge | (default, Jul  1 2019, 22:01:29) [MSC v.1900 64 bit (AMD64)]'

In [66]:
def download_GCM_datashare(scn, var, extent, downPath, outPath, model=None):
    """Descarga los datos de la zona de estudio del sevidor 'https://portal.nccs.nasa.gov/datashare/NEXGDDP/BCSD/' y genera los archivos NetCDF para el escenario, variable y modelo de interés
    
    Parámetros:
    -----------
    scn:       string. Escenario climático: 'historical' para datos históricos, rcp45' o 'rcp85' para escenarios futuros
    var:       string. Variable: 'pr', precipitación; 'tasmax', temperatura máxima del aire en superficie; 'tasmin', temperatura mínima del aire en superficie
    extent:    list [W, E, S, N]. Extensión del área de estudio en grados
    downPath:  string. Ruta donde descargar los datos descargados del servidor
    outPath:   string. Ruta destino donde guardar los archivos generados
    model:     string. Nombre del modelo climático. P.ej.: 'inmcm4', 'bcc-csm1-1', 'NorESM1-M'... Por defecto es 'None' y se descargan todos los modelos disponibles
    
    Salidas:
    --------
    Archivos NetCDF en el directorio 'outPath'
    """
    
    # carpeta de destino
    outPath = outPath + scn + '/' + var + '/'

    # acceder a la url del RCP y variable
    url0 = 'https://portal.nccs.nasa.gov/datashare/NEXGDDP/BCSD/{0}/day/atmos/{1}/r1i1p1/v1.0/'.format(scn, var)
    urllib.request.urlretrieve(url0, 'summary.txt')
    data = open('summary.txt', 'r').readlines()

    # lista con los archivos disponibles para la variable
    files = list()
    for l, line in enumerate(data):
        if model is None:
            if var in line:
                files.append(line[line.find(var):line.find('.nc') + 3])
        else:
            if (var in line) & (model in line):
                files.append(line[line.find(var):line.find('.nc') + 3])
    print('nº de archivos disponibles: {0}'.format(len(files)))

    # descarga del archivo, si no existe ya en la carpeta de destino
    for file in tqdm.tqdm(files):
        if os.path.exists(outPath + file):
            continue
        else:
            print('Descargando: ' + file)
            # 'flags' sobre la descarga
            not_downloaded, nnn = True, 0
            while not_downloaded:
                try:
                    urllib.request.urlretrieve(url0 + file, downPath + file)
                    cdo = 'cdo sellonlatbox,{0},{1},{2},{3}  {4}{5} {6}{5}'.format(*extent, downPath, file, outPath)
                    rm = 'rm {0}{1}'.format(downPath, file)
                    os.system(cdo)
                    os.system(rm)  
                    not_downloaded = False
                except:
                    print('Try again...')
                    nnn += 1
                    if nnn > 15:
                        print('No se ha descargado el fichero: ' + file)
                        break

In [1]:
def download_GCM_thredds(scn, var, model, extent, period, path):
    """Descarga los datos de los modelos climáticos globales (GCM) del servidor THREDDS de la NASA y genera los archivos NetCDF para el escenario, variable y modelo de interés.

    'https://dataserver.nccs.nasa.gov/thredds/ncss/bypass/NEX-GDDP/bcsd/'
    
    Parámetros:
    -----------
    scn:       string. Escenario climático: 'historical' para datos históricos, rcp45' o 'rcp85' para escenarios futuros
    var:       string. Variable: 'pr', precipitación; 'tasmax', temperatura máxima; 'tasmin', temperatura mínma
    model:     string. Modelo climático. P.ej. 'inmcm4.ncml', 'bcc-csm1-1.ncml'...
    extent:    list [N,W,E,S]. Coordenadas geográficas con la extensión de la zona de estudio
    period:    list of int. Años inicial y final para los que descargar los datos
    path:      string. Directorio donde guardar las descargas
    
    Salidas:
    --------
    Un archivo NetCDF para cada año en la carpeta 'path/scn/var/'
    """

    # definir carpeta donde guardar las descargas
    outPath = path + scn + '/' + var + '/'
    if os.path.exists(outPath) == False:
        os.makedirs(outPath)

    # servidor donde se alojan los datos
    server = 'https://dataserver.nccs.nasa.gov/thredds/ncss/bypass/NEX-GDDP/bcsd/'
    
    # comprobar que el periodo es correcto
    if len(period) == 2 and all(isinstance(val, int) for val in period):
        pass
    else:
        print('ERROR. La lista "period" con el perido de estudio es errónea.')
        return
    
    # bucle para la descarga de los datos de cada año hasta 2100
    for year in tqdm.tqdm(range(period[0], period[1] + 1)):
        # fechas de inicio y fin de la descarga
        dates = [datetime(year, 1, 1).date(), datetime(year, 12, 31).date()]

        # nombre del archivo a descargar
        filename = '{0}_day_BCSD_{1}_r1i1p1_{2}_{3}.nc'.format(var, scn, model[:-5],
                                                               dates[0].year)

        if os.path.exists(outPath + filename):
            continue # saltar al siguiente año, si ya existe el archivo
        else:
            # url de acceso a los datos del año
            url = ('{0}{1}/r1i1p1/{2}/' + # RCP y variable
                   '{3}?var={2}&north={4}&west={5}&east={6}&south={7}' + # modelo, variable y extensión
                   '&disableProjSubset=on' + # configuración no modificable
                   '&horizStride=1' + # configuración no modificable
                   '&time_start={8}T12%3A00%3A00Z' + # fecha inicial
                   '&time_end={9}T12%3A00%3A00Z' + # fecha final
                   '&timeStride=1&' +
                   'addLatLon=true').format(server, scn, var, model,*extent,*dates)
            print(url)
            print('Descargando', filename)
            # 'flags' sobre la descarga
            nnn, not_downloaded = 0, True
            while not_downloaded:
                try:
                    # descargar datos
                    urllib.request.urlretrieve(url, outPath + filename)
                    not_downloaded = False
                except:
                    nnn += 1
                    print("Intentando de nuevo... Iteración {0:>2}".format(nnn), end='\r')
                    if nnn > 15:
                        print('No se descarga el modelo:', model)
                        break